In [1]:
import nltk 
# nltk.download('all')   # no need to download all packages, uncomment it for 1st time running code
from nltk.stem import WordNetLemmatizer
import re    # regular expression
import numpy as np
import pandas as pd
from pprint import pprint  # prettier print
import gensim       # Gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

# disable warnings
import warnings
warnings.filterwarnings(action='ignore')

C:\Users\ku-team\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
# load the save the english comments corpus into a pickle file as a list
pickle_in = open("english_comments.pickle","rb")
english_comments = pickle.load(pickle_in)    # loads the list of comments
pprint(english_comments[:1])

['Our stay with Marcus in Bristol was fantastic in every way! He was a great '
 'host - picking us up at the bus stop, recommending places to try, leaving '
 'plenty of pastries and other breakfast items to enjoy in the morning. The '
 'flat itself was modern, bright, clean and spacious - and best of all, right '
 "on Bristol's lovely harbourside. We will definitely stay again next time "
 "we're in Bristol - thanks again Marcus!"]


In [3]:
# put the list into dataframe to view the nature of texts 
df = pd.DataFrame(english_comments,columns=['comments'])
pprint(df.head(5))
print('\n\n')
pprint(df.info())

                                            comments
0  Our stay with Marcus in Bristol was fantastic ...
1  Marcus is a brilliant, warm and friendly host....
2  My mum Angela and I have stayed at Marcus' ama...
3  Marcus was an exceptional host. I only stayed ...
4  Marcus was welcoming, easy going and very help...



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96419 entries, 0 to 96418
Data columns (total 1 columns):
comments    96419 non-null object
dtypes: object(1)
memory usage: 753.4+ KB
None


In [4]:
# tokenize sentences and remove panctuations and emojis
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [5]:
tokenized_texts = list(sent_to_words(english_comments))
print(tokenized_texts[:2])


[['our', 'stay', 'with', 'marcus', 'in', 'bristol', 'was', 'fantastic', 'in', 'every', 'way', 'he', 'was', 'great', 'host', 'picking', 'us', 'up', 'at', 'the', 'bus', 'stop', 'recommending', 'places', 'to', 'try', 'leaving', 'plenty', 'of', 'pastries', 'and', 'other', 'breakfast', 'items', 'to', 'enjoy', 'in', 'the', 'morning', 'the', 'flat', 'itself', 'was', 'modern', 'bright', 'clean', 'and', 'spacious', 'and', 'best', 'of', 'all', 'right', 'on', 'bristol', 'lovely', 'harbourside', 'we', 'will', 'definitely', 'stay', 'again', 'next', 'time', 'we', 're', 'in', 'bristol', 'thanks', 'again', 'marcus'], ['marcus', 'is', 'brilliant', 'warm', 'and', 'friendly', 'host', 'he', 'picked', 'us', 'up', 'from', 'the', 'railway', 'station', 'he', 'took', 'anne', 'to', 'the', 'doctor', 'and', 'drove', 'us', 'around', 'wherever', 'we', 'needed', 'to', 'go', 'in', 'bristol', 'and', 'dropped', 'us', 'back', 'at', 'the', 'railway', 'station', 'when', 'we', 'were', 'leaving', 'his', 'flat', 'is', 'very'

In [21]:
# see the most common and most rare words
from collections import  Counter    # dictionary collection
def get_words_frequency(texts):
    counter = Counter()
    for  doc in texts:
        for word in doc:
            counter[word]+=1
    return counter

word_freq = get_words_frequency(tokenized_texts)
pprint(word_freq) # @ the end of the printed texts, still chinise  words in there.
first10pairs = {k: word_freq[k] for k in sorted(word_freq.items(),key=lambda x: x[1],reverse=True)[:20]}
first10pairs

Counter({'and': 186280,
         'the': 154077,
         'to': 93356,
         'was': 73952,
         'in': 64651,
         'is': 63608,
         'very': 59606,
         'great': 49954,
         'for': 47042,
         'stay': 43553,
         'we': 42815,
         'of': 41167,
         'with': 37414,
         'place': 32685,
         'it': 32402,
         'location': 30508,
         'lovely': 30187,
         'you': 26730,
         'bristol': 26587,
         'clean': 25628,
         'room': 22558,
         'would': 21626,
         'host': 20727,
         'really': 19666,
         'house': 19121,
         'nice': 18825,
         'had': 18530,
         'comfortable': 18213,
         'recommend': 18204,
         'at': 17680,
         'as': 17458,
         'good': 16679,
         'this': 16439,
         'were': 16193,
         'flat': 15887,
         'again': 14698,
         'our': 14624,
         'from': 14350,
         'on': 13921,
         'but': 13856,
         'us': 13653,
         'so'

         'communicating': 265,
         'overlooking': 264,
         'interior': 264,
         'zowie': 264,
         'wife': 263,
         'throw': 263,
         'communicated': 263,
         'promptly': 262,
         'summer': 262,
         'toys': 262,
         'butter': 260,
         'olivia': 260,
         'walks': 259,
         'jill': 259,
         'lucky': 258,
         'furniture': 258,
         'montpelier': 258,
         'eco': 258,
         'ben': 258,
         'jim': 258,
         'vibe': 257,
         'range': 257,
         'nuccia': 255,
         'alberto': 255,
         'cara': 255,
         'general': 254,
         'compact': 254,
         'necessary': 253,
         'tesco': 252,
         'understanding': 251,
         'amazingly': 251,
         'case': 251,
         'doesn': 251,
         'conversation': 250,
         'offering': 250,
         'owners': 250,
         'peter': 249,
         'ashley': 249,
         'exceptional': 248,
         'box': 247,
         'matt

         'hoped': 123,
         'relatively': 123,
         'russ': 123,
         'jason': 123,
         'rain': 122,
         'anyway': 122,
         'missing': 122,
         'electric': 122,
         'budget': 122,
         'cycle': 122,
         'unit': 121,
         'fancy': 121,
         'southville': 121,
         'cheers': 121,
         'showers': 121,
         'vegan': 121,
         'provisions': 121,
         'cons': 121,
         'except': 121,
         'closed': 121,
         'hannah': 121,
         'repeat': 120,
         'memorable': 120,
         'nd': 120,
         'fixed': 120,
         'wharf': 120,
         'turn': 120,
         'departure': 119,
         'worry': 119,
         'towel': 119,
         'frequent': 119,
         'annex': 119,
         'joy': 118,
         'weekends': 118,
         'creative': 118,
         'unusual': 118,
         'replying': 118,
         'theatre': 118,
         'soap': 118,
         'sleeper': 118,
         'positioned': 117,
        

         'elsewhere': 62,
         'boutique': 62,
         'similar': 62,
         'rush': 62,
         'aren': 62,
         'styled': 62,
         'require': 62,
         'salt': 62,
         'chosen': 62,
         'answers': 62,
         'paying': 62,
         'val': 62,
         'clive': 62,
         'pru': 62,
         'eli': 62,
         'marina': 61,
         'airb': 61,
         'accommodations': 61,
         'ticket': 61,
         'conditioner': 61,
         'wee': 61,
         'mobile': 61,
         'initially': 61,
         'busses': 61,
         'josie': 61,
         'colleagues': 61,
         'cups': 61,
         'feature': 61,
         'piano': 61,
         'instant': 61,
         'spec': 61,
         'courteous': 61,
         'bowl': 61,
         'hob': 61,
         'becki': 61,
         'naila': 61,
         'juliette': 61,
         'notes': 60,
         'concerned': 60,
         'rough': 60,
         'approx': 60,
         'respect': 60,
         'shows': 60,
         

         'susan': 37,
         'final': 37,
         'landing': 37,
         'climbing': 37,
         'oliver': 37,
         'club': 37,
         'lauren': 37,
         'nina': 37,
         'zero': 37,
         'fishponds': 37,
         'kirsten': 37,
         'gildas': 37,
         'housemate': 36,
         'safely': 36,
         'mid': 36,
         'display': 36,
         'fabulously': 36,
         'memory': 36,
         'heaven': 36,
         'red': 36,
         'icing': 36,
         'changes': 36,
         'sunset': 36,
         'varied': 36,
         'excited': 36,
         'continue': 36,
         'tenants': 36,
         'pleasing': 36,
         'trust': 36,
         'specially': 36,
         'player': 36,
         'watched': 36,
         'wealth': 36,
         'wanna': 36,
         'zone': 36,
         'yr': 36,
         'smelled': 36,
         'obvious': 36,
         'noting': 36,
         'balloons': 36,
         'drawback': 36,
         'lamp': 36,
         'occupied': 36,
  

...

         'hassel': 1,
         'cheapish': 1,
         'noicy': 1,
         'blank': 1,
         'empting': 1,
         'recyling': 1,
         'anticpated': 1,
         'goop': 1,
         'uneducated': 1,
         'origins': 1,
         'accuse': 1,
         'isis': 1,
         'uglygood': 1,
         'centralable': 1,
         'officebadvery': 1,
         'uglyroad': 1,
         'blaring': 1,
         'thickness': 1,
         'disagreed': 1,
         'unproblematic': 1,
         'inset': 1,
         'highs': 1,
         'unpleasantly': 1,
         'eyemask': 1,
         'streetlight': 1,
         'litteraly': 1,
         'recreational': 1,
         'stingy': 1,
         'helicopter': 1,
         'whiniest': 1,
         'broadcast': 1,
         'intolerable': 1,
         'wink': 1,
         'sh': 1,
         'explanatory': 1,
         'thanksmarek': 1,
         'darkened': 1,
         'slicker': 1,
         'laundrymat': 1,
         'touchpads': 1,
         'superquicly': 1,
        

         'overstayed': 1,
         'desighned': 1,
         'kharkiv': 1,
         'ukraine': 1,
         'owh': 1,
         'doughy': 1,
         'translation': 1,
         'cleanclose': 1,
         'culdasac': 1,
         'belinda': 1,
         'bunglalow': 1,
         'mahen': 1,
         'garth': 1,
         'sentiments': 1,
         'letterbox': 1,
         'equippedlovely': 1,
         'providedin': 1,
         'downn': 1,
         'lance': 1,
         'centrehighly': 1,
         'mexicana': 1,
         'researched': 1,
         'splayed': 1,
         'lest': 1,
         'shabbier': 1,
         'disrepair': 1,
         'ysb': 1,
         'marsh': 1,
         'swell': 1,
         'squared': 1,
         'roxys': 1,
         'exceded': 1,
         'wss': 1,
         'highlighy': 1,
         'beholder': 1,
         'shanks': 1,
         'pony': 1,
         'versha': 1,
         'sarahis': 1,
         'centresarah': 1,
         'tablebthough': 1,
         'remotes': 1,
         'quiet

         'hazards': 1,
         'carseats': 1,
         'bristolcity': 1,
         'joyous': 1,
         'helpes': 1,
         'lucious': 1,
         'patrolled': 1,
         'hosthighly': 1,
         'lorraines': 1,
         'homlier': 1,
         'cleanedwe': 1,
         'planningi': 1,
         'tiresome': 1,
         'samirbis': 1,
         'replayed': 1,
         'iroom': 1,
         'pus': 1,
         'gao': 1,
         'obtainable': 1,
         'checkins': 1,
         'pollys': 1,
         'chocked': 1,
         'juno': 1,
         'graphic': 1,
         'letterpress': 1,
         '非常漂亮且大且设施齐全': 1,
         '这个价钱值得': 1,
         '唯一不好的地方就是位置在山上': 1,
         '要爬超大的坡': 1,
         'pica': 1,
         'passionately': 1,
         'tests': 1,
         'mims': 1,
         'brookes': 1,
         'spatulas': 1,
         'deckchairs': 1,
         'drys': 1,
         'chuck': 1,
         'stayin': 1,
         'waived': 1,
         'oscars': 1,
         'moonlight': 1,
         'aftered':

         'defienietly': 1,
         'overused': 1,
         'cosyclean': 1,
         'wouldhighly': 1,
         'depths': 1,
         'aheadi': 1,
         'deffinantly': 1,
         'ransacked': 1,
         'declining': 1,
         'farcical': 1,
         'ensued': 1,
         'perfunctory': 1,
         'uttered': 1,
         'entailed': 1,
         'stipulation': 1,
         'uncanny': 1,
         'baffled': 1,
         'friendlynature': 1,
         'nicegreat': 1,
         'commination': 1,
         'plying': 1,
         'angled': 1,
         'government': 1,
         'popti': 1,
         'mina': 1,
         'superdog': 1,
         'piping': 1,
         'nescafe': 1,
         'dolce': 1,
         'fastidious': 1,
         'noses': 1,
         'sniff': 1,
         'sturminster': 1,
         'iut': 1,
         'hangerscar': 1,
         'experence': 1,
         'coarse': 1,
         'meed': 1,
         'deff': 1,
         'helpers': 1,
         'lincolnshire': 1,
         'cyclical': 1

{('and', 186280): 0,
 ('the', 154077): 0,
 ('to', 93356): 0,
 ('was', 73952): 0,
 ('in', 64651): 0,
 ('is', 63608): 0,
 ('very', 59606): 0,
 ('great', 49954): 0,
 ('for', 47042): 0,
 ('stay', 43553): 0,
 ('we', 42815): 0,
 ('of', 41167): 0,
 ('with', 37414): 0,
 ('place', 32685): 0,
 ('it', 32402): 0,
 ('location', 30508): 0,
 ('lovely', 30187): 0,
 ('you', 26730): 0,
 ('bristol', 26587): 0,
 ('clean', 25628): 0}

In [7]:
# to filter chinese words or get only english words
def get_english_words(documents):
    ''' filters out any non-english words'''
    english_words = [[ w for w in doc if re.match(r'[A-Z, a-z]', w)] for doc in documents]
    return english_words


fitered_english_words = get_english_words(tokenized_texts)
print(fitered_english_words[:2])

[['our', 'stay', 'with', 'marcus', 'in', 'bristol', 'was', 'fantastic', 'in', 'every', 'way', 'he', 'was', 'great', 'host', 'picking', 'us', 'up', 'at', 'the', 'bus', 'stop', 'recommending', 'places', 'to', 'try', 'leaving', 'plenty', 'of', 'pastries', 'and', 'other', 'breakfast', 'items', 'to', 'enjoy', 'in', 'the', 'morning', 'the', 'flat', 'itself', 'was', 'modern', 'bright', 'clean', 'and', 'spacious', 'and', 'best', 'of', 'all', 'right', 'on', 'bristol', 'lovely', 'harbourside', 'we', 'will', 'definitely', 'stay', 'again', 'next', 'time', 'we', 're', 'in', 'bristol', 'thanks', 'again', 'marcus'], ['marcus', 'is', 'brilliant', 'warm', 'and', 'friendly', 'host', 'he', 'picked', 'us', 'up', 'from', 'the', 'railway', 'station', 'he', 'took', 'anne', 'to', 'the', 'doctor', 'and', 'drove', 'us', 'around', 'wherever', 'we', 'needed', 'to', 'go', 'in', 'bristol', 'and', 'dropped', 'us', 'back', 'at', 'the', 'railway', 'station', 'when', 'we', 'were', 'leaving', 'his', 'flat', 'is', 'very'

In [8]:
# see words and their frequency
words_freq=  get_words_frequency(fitered_english_words)
first10pairs = {k: words_freq[k] for k in sorted(words_freq.keys())[:20]}
first10pairs

{'aa': 4,
 'aaalso': 1,
 'aagain': 1,
 'aardman': 1,
 'aardvark': 1,
 'aaron': 3,
 'aasy': 1,
 'ab': 4,
 'aback': 2,
 'abandoned': 5,
 'abase': 1,
 'abb': 3,
 'abbey': 11,
 'abbeywood': 1,
 'abbi': 12,
 'abbiamo': 1,
 'abbie': 3,
 'abbots': 1,
 'abby': 2,
 'abc': 2}

In [9]:
# Check the most common or frequent words
most_common_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
most_common_words[:20]   # first 20 words

[('and', 186280),
 ('the', 154077),
 ('to', 93356),
 ('was', 73952),
 ('in', 64651),
 ('is', 63608),
 ('very', 59606),
 ('great', 49954),
 ('for', 47042),
 ('stay', 43553),
 ('we', 42815),
 ('of', 41167),
 ('with', 37414),
 ('place', 32685),
 ('it', 32402),
 ('location', 30508),
 ('lovely', 30187),
 ('you', 26730),
 ('bristol', 26587),
 ('clean', 25628)]

In [10]:
# see the rare words 
rare_words = sorted(word_freq.items(), key=lambda x: x[1])   # NB: word_freq is a dictionary colloction
pprint(rare_words[:10])  #  top 10 rare words

[('chauffeur', 1),
 ('ireland', 1),
 ('foreward', 1),
 ('boatload', 1),
 ('markus', 1),
 ('cellular', 1),
 ('hovered', 1),
 ('saunter', 1),
 ('arranges', 1),
 ('availablity', 1)]


In [22]:
# Build the bigram & trigram models  [the model creates 2 or 3 wordy, most common occuring  phares]
bigram = gensim.models.Phrases(fitered_english_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[fitered_english_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[fitered_english_words[0]]])

['our', 'stay', 'with', 'marcus', 'in', 'bristol', 'was', 'fantastic', 'in', 'every', 'way', 'he', 'was', 'great', 'host', 'picking', 'us', 'up', 'at', 'the', 'bus', 'stop', 'recommending', 'places', 'to', 'try', 'leaving', 'plenty', 'of', 'pastries', 'and', 'other', 'breakfast', 'items', 'to', 'enjoy', 'in', 'the', 'morning', 'the', 'flat', 'itself', 'was', 'modern', 'bright', 'clean', 'and', 'spacious', 'and', 'best', 'of', 'all', 'right', 'on', 'bristol', 'lovely', 'harbourside', 'we', 'will', 'definitely', 'stay', 'again', 'next', 'time', 'we', 're', 'in', 'bristol', 'thanks', 'again', 'marcus']


In [12]:
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
print('number of stop words:',len(stop_words))

# define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatize_texts(texts):
    lemmatizer = WordNetLemmatizer()
    data_out =[]
    for doc in texts:
            data_out.append([lemmatizer.lemmatize(word) for word in doc])
    return data_out 

number of stop words: 179


In [13]:
# remove the stopwords
data_words_nostops = remove_stopwords(fitered_english_words)
print("after removing stop words:",data_words_nostops[0])

# create bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
print("\nbigram data words:",data_words_bigrams[0])

#  lemmatize nouns
lemmatized_data = lemmatize_texts(data_words_bigrams)
print('\n\nAfter lemmatization:',lemmatized_data[:1])

after removing stop words: ['stay', 'marcus', 'bristol', 'fantastic', 'every', 'way', 'great', 'host', 'picking', 'us', 'bus', 'stop', 'recommending', 'places', 'try', 'leaving', 'plenty', 'pastries', 'breakfast', 'items', 'enjoy', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'best', 'right', 'bristol', 'lovely', 'harbourside', 'definitely', 'stay', 'next', 'time', 'bristol', 'thanks', 'marcus']

bigram data words: ['stay', 'marcus', 'bristol', 'fantastic', 'every', 'way', 'great', 'host', 'picking', 'us', 'bus', 'stop', 'recommending', 'places', 'try', 'leaving', 'plenty', 'pastries', 'breakfast', 'items', 'enjoy', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'best', 'right', 'bristol', 'lovely', 'harbourside', 'definitely', 'stay', 'next', 'time', 'bristol', 'thanks', 'marcus']


After lemmatization: [['stay', 'marcus', 'bristol', 'fantastic', 'every', 'way', 'great', 'host', 'picking', 'u', 'bus', 'stop', 'recommending', 'place', 'try', 'leaving', 'plen

In [14]:
# after removing stop words or pre-processing : check the most common and very the rare words
wf = get_words_frequency(lemmatized_data)
most_common_words = sorted(wf.items(), key=lambda x: x[1], reverse=True)
most_common_words[:20]   # top 20 most common words

[('great', 49961),
 ('stay', 43840),
 ('place', 35320),
 ('location', 30625),
 ('lovely', 30188),
 ('host', 28772),
 ('bristol', 26618),
 ('clean', 25632),
 ('room', 23979),
 ('would', 21626),
 ('really', 19666),
 ('house', 19255),
 ('nice', 18826),
 ('comfortable', 18213),
 ('recommend', 18204),
 ('good', 16698),
 ('flat', 16037),
 ('u', 13653),
 ('home', 13089),
 ('everything', 12855)]

In [15]:
# after removing stop words or pre-processing : check the rare words
wf = get_words_frequency(lemmatized_data)
rare_words = sorted(wf.items(), key=lambda x: x[1])
pprint(rare_words[:20])   # first 20 words

[('chauffeur', 1),
 ('ireland', 1),
 ('foreward', 1),
 ('markus', 1),
 ('cellular', 1),
 ('hovered', 1),
 ('saunter', 1),
 ('arranges', 1),
 ('availablity', 1),
 ('macus', 1),
 ('beet', 1),
 ('fraternity', 1),
 ('spotlesly', 1),
 ('guast', 1),
 ('vegfest', 1),
 ('deny', 1),
 ('wecolming', 1),
 ('strives', 1),
 ('morethe', 1),
 ('lovliness', 1)]


In [16]:
def remove_rare_words(texts,words_freq):
    '''removes the rare words'''
    words_data = [[w for w in doc if words_freq[w]>7] for doc in texts]
    return words_data


In [17]:
# remove the rare words from texts
words_data = remove_rare_words(lemmatized_data,wf)
pprint(words_data[:10])

[['stay',
  'marcus',
  'bristol',
  'fantastic',
  'every',
  'way',
  'great',
  'host',
  'picking',
  'u',
  'bus',
  'stop',
  'recommending',
  'place',
  'try',
  'leaving',
  'plenty',
  'pastry',
  'breakfast',
  'item',
  'enjoy',
  'morning',
  'flat',
  'modern',
  'bright',
  'clean',
  'spacious',
  'best',
  'right',
  'bristol',
  'lovely',
  'harbourside',
  'definitely',
  'stay',
  'next',
  'time',
  'bristol',
  'thanks',
  'marcus'],
 ['marcus',
  'brilliant',
  'warm',
  'friendly',
  'host',
  'picked',
  'u',
  'railway_station',
  'took',
  'anne',
  'doctor',
  'drove',
  'u',
  'around',
  'wherever',
  'needed',
  'go',
  'bristol',
  'dropped',
  'u',
  'back',
  'railway_station',
  'leaving',
  'flat',
  'modern',
  'comfortable',
  'clean',
  'well',
  'heated',
  'marcus',
  'provided',
  'u',
  'everything',
  'could',
  'wish',
  'wish',
  'could',
  'stayed',
  'longer'],
 ['mum',
  'angela',
  'stayed',
  'marcus',
  'amazing',
  'apartment',
  'tw

In [18]:
# create dictionary
id2word = corpora.Dictionary(words_data)
print("number of keys:",len(id2word.keys()))
# view the first word of the dictionary
print(id2word[0])   # see word with key =0

# create corpus
corpus_texts= words_data
print(corpus_texts[:3])

# bag of words or vector spaces
corpus = [id2word.doc2bow(text) for text in corpus_texts]
print(corpus[:2])

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

number of keys: 5785
best
[['stay', 'marcus', 'bristol', 'fantastic', 'every', 'way', 'great', 'host', 'picking', 'u', 'bus', 'stop', 'recommending', 'place', 'try', 'leaving', 'plenty', 'pastry', 'breakfast', 'item', 'enjoy', 'morning', 'flat', 'modern', 'bright', 'clean', 'spacious', 'best', 'right', 'bristol', 'lovely', 'harbourside', 'definitely', 'stay', 'next', 'time', 'bristol', 'thanks', 'marcus'], ['marcus', 'brilliant', 'warm', 'friendly', 'host', 'picked', 'u', 'railway_station', 'took', 'anne', 'doctor', 'drove', 'u', 'around', 'wherever', 'needed', 'go', 'bristol', 'dropped', 'u', 'back', 'railway_station', 'leaving', 'flat', 'modern', 'comfortable', 'clean', 'well', 'heated', 'marcus', 'provided', 'u', 'everything', 'could', 'wish', 'wish', 'could', 'stayed', 'longer'], ['mum', 'angela', 'stayed', 'marcus', 'amazing', 'apartment', 'two', 'week', 'august', 'relocating', 'bristol', 'lovely', 'experience', 'host', 'apartment', 'extremely', 'confortable', 'located', 'nice', '

[[('best', 1),
  ('breakfast', 1),
  ('bright', 1),
  ('bristol', 3),
  ('bus', 1),
  ('clean', 1),
  ('definitely', 1),
  ('enjoy', 1),
  ('every', 1),
  ('fantastic', 1),
  ('flat', 1),
  ('great', 1),
  ('harbourside', 1),
  ('host', 1),
  ('item', 1),
  ('leaving', 1),
  ('lovely', 1),
  ('marcus', 2),
  ('modern', 1),
  ('morning', 1),
  ('next', 1),
  ('pastry', 1),
  ('picking', 1),
  ('place', 1),
  ('plenty', 1),
  ('recommending', 1),
  ('right', 1),
  ('spacious', 1),
  ('stay', 2),
  ('stop', 1),
  ('thanks', 1),
  ('time', 1),
  ('try', 1),
  ('u', 1),
  ('way', 1)]]

How does LDA work or converge?
`
For each document d, compute P( topic t | document d ) := proportion of words in document d that are assigned to topic t
For each topic t, P( word w | topic t ) := proportion of assignments to topic t that come from word w (across all documents)
For each word w, reassign topic t’, where we choose topic t’ with probability P( topic t’ | word w ) = P( topic t’ | document d ) * P( word w | topic t’ )`

In [19]:
import time
start_time = time.time()
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

done_time = time.time()

pprint(" %.3f secs" % (done_time - start_time))

' 299.855 secs'


In [20]:
# Print the 10 keyword in topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.249*"nice" + 0.219*"good" + 0.062*"value" + 0.043*"shower" + '
  '0.040*"people" + 0.036*"money" + 0.012*"window" + 0.010*"service" + '
  '0.010*"drink" + 0.010*"load"'),
 (1,
  '0.077*"bit" + 0.044*"milk" + 0.040*"booking" + 0.030*"bread" + '
  '0.030*"heating" + 0.025*"another" + 0.023*"fridge" + 0.022*"outstanding" + '
  '0.021*"inside" + 0.020*"others"'),
 (2,
  '0.104*"clean" + 0.094*"lovely" + 0.072*"really" + 0.060*"room" + '
  '0.058*"house" + 0.056*"comfortable" + 0.037*"bed" + 0.034*"home" + '
  '0.030*"friendly" + 0.026*"communication"'),
 (3,
  '0.089*"coffee" + 0.071*"couple" + 0.063*"tea" + 0.039*"especially" + '
  '0.033*"group" + 0.033*"sofa" + 0.024*"decor" + 0.024*"wanted" + '
  '0.023*"basic" + 0.023*"slept"'),
 (4,
  '0.049*"well" + 0.045*"easy" + 0.041*"city" + 0.040*"space" + 0.038*"centre" '
  '+ 0.032*"area" + 0.030*"close" + 0.030*"walk" + 0.028*"quiet" + '
  '0.027*"check"'),
 (5,
  '0.054*"u" + 0.035*"also" + 0.032*"kitchen" + 0.024*"could" + 0.024*

` NB: next steps (a)further preprocesing texts,like removing the firsrt few most common words,(b)interpretation of the topics,(c)visualizing the topics clusters, and (d) documents ranking...`
--------------------
`references`: gensim documentation 